In [2]:
%store -r jhj_emoji_texts
%store -r jhj_emoji_labels

In [3]:
#hashtags und emojis sind entfernt worden (in der corpusfile)
#preprocessing wird hier gemacht

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [5]:
#######clean#######

In [6]:
from num2words import num2words
#convert everything to lowercase
jhj_emoji_texts = [x.lower() for x in jhj_emoji_texts]
#exchange numbers with their string representation
#make numbers to text
temp = []
without_num = []
for post in jhj_emoji_texts:
    for word in post.split(" "):
        if word.isdigit():
            #print(word)
            word = num2words(word, lang="de")
        temp.append(word)
    without_num.append((' '.join(temp)))
    temp = []
jhj_emoji_texts = without_num

In [7]:
#######feature extraction#########

In [8]:
#without deleting stopwords
#ngram feature matrices

#unigram
uni_vectorizer = CountVectorizer(ngram_range=(1,1))
unigram_feature_vector = uni_vectorizer.fit_transform(jhj_emoji_texts).toarray() 

#uni and bigram
uni_bi_vectorizer = CountVectorizer(ngram_range=(1,2))
unigram_bi_feature_vector = uni_bi_vectorizer.fit_transform(jhj_emoji_texts).toarray() 

#bigram
bi_vectorizer = CountVectorizer(ngram_range=(2,2))
bigram_feature_vector = bi_vectorizer.fit_transform(jhj_emoji_texts).toarray()


#trigram
tri_vectorizer = CountVectorizer(ngram_range=(3,3))
trigram_feature_vector = tri_vectorizer.fit_transform(jhj_emoji_texts).toarray() 

In [9]:
#tfidf vectorizer ohne stopwords zu löschen
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(jhj_emoji_texts).toarray() #feature matrix

In [10]:
#numerical features
#average length of post, average sentence length, average number of punctuation
#punctuation weg lassen weil nicht wirklich aussagekräftig


import numpy
import string
from nltk import tokenize

post_length = [] #feature vector
counter = 0
number_of_punct = [] #feature vector
sentence_lengths = [] 
tokenized_posts = []
for text in jhj_emoji_texts:
    post_length.append([len(text.split())])
    tokenized_posts.append(tokenize.sent_tokenize(text))
    for char in text:
        if char in string.punctuation:
            counter+=1
    number_of_punct.append([counter])
    counter = 0
sent = []
for post in tokenized_posts:
    for sentence in post:
        sent.append(len(sentence.split()))
    sentence_lengths.append(sent)
    sent = []
#get average
average_sent_len = [] #feature vector
for post in sentence_lengths:
    average_sent_len.append([sum(post)/len(post)])    
print(len(sentence_lengths))


858


In [12]:
#POS features
import spacy
nlp = spacy.load('de_core_news_md')

#get POStags via spacy
spacy_obj = []
post_as_pos_tags = []
as_pos = [] #this
for text in jhj_emoji_texts:
    spacy_obj.append(nlp(text))
for text in spacy_obj:
    for token in text:
        post_as_pos_tags.append(token.tag_)
    as_pos.append(' '.join(post_as_pos_tags))
    post_as_pos_tags = []

#neue POS-features
ADV_count = []
PPER_count = []
comma_count = []
NE_count = []
NN_count = []
ADJD_count = []
for post in as_pos:
    ADV_count.append([post.count("ADV")])
    PPER_count.append([post.count("PPER")])
    comma_count.append([post.count("$,")])
    NE_count.append([post.count("NE")])
    NN_count.append([post.count("NN")])
    ADJD_count.append([post.count("ADJD")])
    
print(len(ADJD_count))

#EXPERIMENTS
#get posts as sequence of pos tags
#as_pos = sequence of pos tags
#make sequences into count vectorizers
pos_vec_uni = CountVectorizer(ngram_range=(1,1))
pos_uni_count_vec = pos_vec_uni.fit_transform(as_pos).toarray() #uni feature vector
pos_vec_bi = CountVectorizer(ngram_range=(2,2))
pos_bi_count_vec = pos_vec_bi.fit_transform(as_pos).toarray() #bi feature vector
#make theminto tfidf vectorizers
tfidf_vectorizer_pos = TfidfVectorizer()
tfidf_pos = tfidf_vectorizer_pos.fit_transform(as_pos) #tfidf pos feature matrix hat das iwelche benefits??

858


In [13]:
#document vectors as features

spacy_posts = []
vector_posts = []
temp_post = []
for post in jhj_emoji_texts:
    spacy_posts.append(nlp(post))
for post in spacy_posts:
    for word in post:
        temp_post.append(word.vector)
    vector_posts.append(temp_post)
    temp_post = []
    
mean_vector_posts = []
for post_list in vector_posts:
    #print(len(post))
    mean_vector_posts.append([sum(post_list)/len(post_list)])
    
word_vector_feature = numpy.concatenate(mean_vector_posts, axis=0)
print(word_vector_feature.shape)   


(858, 300)


In [14]:
#wordvectors as features

#make all to numpy arrays
np_posts = []
for p in vector_posts:
    np_posts.append(numpy.array(p))
       
#test
padded_arrays = []
for p in np_posts:
       padded_arrays.append(numpy.pad(p, ((0, (55 - p.shape[0])), (0, 0)), 'constant', constant_values=0))
print(len(padded_arrays))


np_pad_arr = numpy.array(padded_arrays)
np_pad_arr.shape #shape (97, 48, 300)

nodoc = numpy.reshape (np_pad_arr, (858, -1)) #FEATURE VECTOR FOR WORDVECTORS
print("from ", np_pad_arr.shape, " to ", nodoc.shape)#just a test for the right dimensions


858
from  (858, 55, 300)  to  (858, 16500)


In [15]:
###################combine feature-vectors:##########################

In [16]:
#combine feature-vectors:

#uni-grams
#unigram_feature_vector

#bigrams
#bigram_feature_vector

#trigrams
#trigram_feature_vector

#unigrams and bigrams
#unigram_bigram_feature_vector

#tfidf

#alle POS-features
pos = numpy.append(ADV_count, comma_count, 1)
pos = numpy.append(pos, NE_count, 1)
pos = numpy.append(pos, NN_count, 1)
pos = numpy.append(pos, ADJD_count, 1)
pos = numpy.append(pos, PPER_count, 1)

#numerical features
#sentence_length
post_sent = numpy.append(post_length, average_sent_len, 1)
post_sent_punct = numpy.append(post_sent, number_of_punct, 1)

#documents vectors
#word_vector_feature

#wordvectors
#nodoc



# 3) uni-grams, bigrams, trigrams
uni_bi_tri_feat_vec = numpy.append(unigram_feature_vector, bigram_feature_vector, 1)
uni_bi_tri_feat_vec = numpy.append(uni_bi_tri_feat_vec,trigram_feature_vector, 1)

# 4) tfidf and bigrams
tfidf_bigram = numpy.append(tfidf, bigram_feature_vector, 1)

# 5) unigrams and POS features
uni_pos_new = numpy.append(unigram_feature_vector, pos, 1)

# 6) unigrams and numerical features
uni_num = numpy.append(unigram_feature_vector, post_sent_punct, 1)

# 7) unigrams and POS tags and numerical features
uni_num_pos = numpy.append(uni_num, pos, 1)

# 8) Bigrams and POS
bi_pos_new = numpy.append(bigram_feature_vector, pos, 1)

# 9) bigrams and numerical features
bi_num = numpy.append(bigram_feature_vector, post_sent_punct, 1)

# 10) bigrams and POS tags and numerical features
bi_num_pos = numpy.append(bi_num, pos, 1)

# 11) tfidf and POS
tfidf_pos_new = numpy.append(tfidf, pos, 1)

# 12) tfidf and numerical features
tfidf_num = numpy.append(tfidf, post_sent_punct, 1)

# 13) tfidf and POS tags and numerical features
tfidf_num_pos = numpy.append(tfidf_num, pos, 1)

# 14) wordvec and POS
wordvec_pos = numpy.append(nodoc, pos, 1)

# 15) wordvec and numerical features
wordvec_num = numpy.append(nodoc, post_sent_punct, 1)

# 16) wordvec and POS tags and numerical features
wordvec_num_pos = numpy.append(wordvec_num, pos, 1)

# 17) docvec and POS
docvec_pos = numpy.append(word_vector_feature, pos, 1)

# 18) docvec and numerical features
docvec_num = numpy.append(word_vector_feature, post_sent_punct, 1)

# 19) docvec and POS tags and numerical features
docvec_num_pos = numpy.append(docvec_num, pos, 1)


# 20)POS and numerical features
pos_post_sent_len = numpy.append(pos, post_sent, 1)
pos_num = numpy.append(post_sent_punct, pos, 1)

# 21) alle
all_features = numpy.append(unigram_feature_vector, bigram_feature_vector, 1)
all_features = numpy.append(all_features, trigram_feature_vector , 1)
all_features = numpy.append(all_features, tfidf, 1)
all_features = numpy.append(all_features,post_sent_punct, 1)
all_features = numpy.append(all_features, pos, 1)
all_features = numpy.append(all_features,nodoc, 1)
all_features = numpy.append(all_features,word_vector_feature, 1)


In [17]:
#########train and test######################

In [18]:
######logistic regression########

In [19]:
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_predict

In [65]:
X = unigram_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))


scores: [0.8372093  0.76744186 0.88372093 0.79069767 0.8372093  0.80232558
 0.77906977 0.80232558 0.84705882 0.81176471]
Accuracy: 0.816 (0.033)


In [66]:
X = bigram_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.77906977 0.74418605 0.8372093  0.72093023 0.87209302 0.77906977
 0.8372093  0.79069767 0.85882353 0.85882353]
Accuracy: 0.808 (0.050)


In [70]:
X = trigram_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.72093023 0.76744186 0.77906977 0.73255814 0.74418605 0.74418605
 0.77906977 0.72093023 0.78823529 0.77647059]
Accuracy: 0.755 (0.024)


In [73]:
X = tfidf#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.87209302 0.8255814  0.88372093 0.79069767 0.81395349 0.81395349
 0.80232558 0.8255814  0.87058824 0.87058824]
Accuracy: 0.837 (0.032)


In [74]:
X = post_sent_punct#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.58139535 0.63953488 0.53488372 0.53488372 0.52325581 0.58139535
 0.47674419 0.47674419 0.57647059 0.56470588]
Accuracy: 0.549 (0.048)


In [75]:
X = pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.59302326 0.48837209 0.46511628 0.56976744 0.5        0.5
 0.54651163 0.54651163 0.58823529 0.55294118]
Accuracy: 0.535 (0.042)


In [76]:
X = nodoc#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8372093  0.74418605 0.76744186 0.74418605 0.80232558 0.79069767
 0.77906977 0.84883721 0.83529412 0.83529412]
Accuracy: 0.798 (0.037)


In [77]:
X = word_vector_feature#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8372093  0.77906977 0.8372093  0.8372093  0.79069767 0.86046512
 0.81395349 0.79069767 0.88235294 0.81176471]
Accuracy: 0.824 (0.031)


In [ ]:
##combinations##

In [78]:
#2
X = unigram_bi_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8372093  0.76744186 0.89534884 0.8255814  0.84883721 0.8255814
 0.81395349 0.81395349 0.85882353 0.81176471]
Accuracy: 0.830 (0.032)


In [79]:
#3
X = uni_bi_tri_feat_vec#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8255814  0.79069767 0.90697674 0.79069767 0.84883721 0.8372093
 0.80232558 0.77906977 0.84705882 0.84705882]
Accuracy: 0.828 (0.037)


In [81]:
#4
X = tfidf_bigram#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.79069767 0.75581395 0.8372093  0.75581395 0.88372093 0.79069767
 0.84883721 0.81395349 0.85882353 0.87058824]
Accuracy: 0.821 (0.044)


In [82]:
#5
X = uni_pos_new#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.86046512 0.80232558 0.88372093 0.76744186 0.87209302 0.77906977
 0.77906977 0.80232558 0.84705882 0.83529412]
Accuracy: 0.823 (0.040)


In [83]:
#6
X = uni_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8372093  0.80232558 0.86046512 0.81395349 0.8372093  0.8372093
 0.79069767 0.76744186 0.83529412 0.84705882]
Accuracy: 0.823 (0.027)


In [84]:
#7
X = uni_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.81395349 0.81395349 0.84883721 0.8255814  0.86046512 0.81395349
 0.76744186 0.77906977 0.84705882 0.83529412]
Accuracy: 0.821 (0.028)


In [85]:
#8
X = bi_pos_new#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.77906977 0.69767442 0.8255814  0.77906977 0.84883721 0.77906977
 0.84883721 0.77906977 0.83529412 0.84705882]
Accuracy: 0.802 (0.046)


In [86]:
#9
X = bi_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.76744186 0.76744186 0.80232558 0.75581395 0.87209302 0.77906977
 0.76744186 0.76744186 0.85882353 0.84705882]
Accuracy: 0.798 (0.042)


In [87]:
#10
X = bi_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.81395349 0.73255814 0.8372093  0.76744186 0.84883721 0.79069767
 0.77906977 0.73255814 0.82352941 0.81176471]
Accuracy: 0.794 (0.039)


In [145]:
#11
X = tfidf_pos_new#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.84883721 0.72093023 0.87209302 0.76744186 0.81395349 0.80232558
 0.80232558 0.80232558 0.87058824 0.83529412]
Accuracy: 0.814 (0.044)


In [89]:
#12
X = tfidf_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8255814  0.81395349 0.81395349 0.79069767 0.8372093  0.80232558
 0.74418605 0.76744186 0.81176471 0.78823529]
Accuracy: 0.800 (0.026)


In [90]:
#13
X = tfidf_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.84883721 0.76744186 0.84883721 0.76744186 0.8255814  0.79069767
 0.69767442 0.76744186 0.83529412 0.81176471]
Accuracy: 0.796 (0.045)


In [91]:
#14
X = wordvec_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8372093  0.75581395 0.80232558 0.76744186 0.80232558 0.80232558
 0.77906977 0.84883721 0.84705882 0.87058824]
Accuracy: 0.811 (0.036)


In [93]:
#15
X = wordvec_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.81395349 0.74418605 0.79069767 0.74418605 0.81395349 0.79069767
 0.77906977 0.84883721 0.84705882 0.83529412]
Accuracy: 0.801 (0.036)


In [94]:
#16
X = wordvec_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.79069767 0.73255814 0.80232558 0.74418605 0.80232558 0.79069767
 0.76744186 0.8372093  0.83529412 0.83529412]
Accuracy: 0.794 (0.035)


In [95]:
#17
X = docvec_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8255814  0.80232558 0.8372093  0.8372093  0.8372093  0.86046512
 0.81395349 0.81395349 0.83529412 0.84705882]
Accuracy: 0.831 (0.016)


In [20]:
#18
X = docvec_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8255814  0.81395349 0.8255814  0.84883721 0.84883721 0.88372093
 0.77906977 0.81395349 0.85882353 0.83529412]
Accuracy: 0.833 (0.027)


In [97]:
#19
X = docvec_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8255814  0.79069767 0.8255814  0.84883721 0.8255814  0.88372093
 0.80232558 0.8255814  0.85882353 0.83529412]
Accuracy: 0.832 (0.025)


In [98]:
#20
X = pos_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.69767442 0.63953488 0.6744186  0.63953488 0.63953488 0.65116279
 0.56976744 0.58139535 0.67058824 0.54117647]
Accuracy: 0.630 (0.048)


In [99]:
#21
X = all_features#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8372093  0.75581395 0.80232558 0.76744186 0.81395349 0.8255814
 0.76744186 0.8372093  0.85882353 0.83529412]
Accuracy: 0.810 (0.034)


In [100]:
##############naive bayes##############

In [21]:
from sklearn.naive_bayes import GaussianNB

In [103]:

X = unigram_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.6627907
 0.69767442 0.6627907  0.72941176 0.69411765]
Accuracy: 0.681 (0.035)


In [105]:

X = bigram_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6744186  0.69767442 0.70930233 0.6744186  0.60465116 0.61627907
 0.63953488 0.68604651 0.6        0.64705882]
Accuracy: 0.655 (0.037)


In [107]:

X = trigram_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.36046512 0.37209302 0.43023256 0.44186047 0.36046512 0.44186047
 0.34883721 0.41860465 0.34117647 0.4       ]
Accuracy: 0.392 (0.037)


In [108]:

X = tfidf#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.65116279
 0.70930233 0.6744186  0.72941176 0.70588235]
Accuracy: 0.683 (0.036)


In [114]:
X = post_sent_punct#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.72093023 0.75581395 0.75581395 0.6627907  0.73255814 0.73255814
 0.74418605 0.69767442 0.78823529 0.75294118]
Accuracy: 0.734 (0.033)


In [115]:
X = pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.62790698 0.65116279 0.73255814 0.61627907 0.6627907  0.58139535
 0.62790698 0.61627907 0.68235294 0.75294118]
Accuracy: 0.655 (0.051)


In [116]:
X = nodoc#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.34883721 0.30232558 0.29069767 0.37209302 0.27906977 0.3255814
 0.31395349 0.39534884 0.28235294 0.74117647]
Accuracy: 0.365 (0.131)


In [117]:
##nice
X = word_vector_feature#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.86046512 0.6627907  0.75581395 0.84883721 0.79069767 0.8255814
 0.79069767 0.73255814 0.83529412 0.8       ]
Accuracy: 0.790 (0.057)


In [113]:
##combinations##

In [118]:
#2
X = unigram_bi_feature_vector#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.76744186 0.74418605 0.74418605 0.74418605 0.68604651 0.68604651
 0.81395349 0.70930233 0.76470588 0.74117647]
Accuracy: 0.740 (0.037)


In [119]:
#3
X = uni_bi_tri_feat_vec#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.79069767 0.75581395 0.75581395 0.73255814 0.72093023 0.69767442
 0.81395349 0.69767442 0.76470588 0.72941176]
Accuracy: 0.746 (0.036)


In [120]:
#4
X = tfidf_bigram#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.76744186 0.70930233 0.73255814 0.70930233 0.70930233 0.65116279
 0.75581395 0.72093023 0.75294118 0.74117647]
Accuracy: 0.725 (0.032)


In [121]:
#5
X = uni_pos_new#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.6627907
 0.69767442 0.6627907  0.72941176 0.69411765]
Accuracy: 0.681 (0.035)


In [122]:
#6
X = uni_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.6627907
 0.69767442 0.6627907  0.72941176 0.69411765]
Accuracy: 0.681 (0.035)


In [123]:
#7
X = uni_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.6627907
 0.69767442 0.6627907  0.72941176 0.69411765]
Accuracy: 0.681 (0.035)


In [124]:
#8
X = bi_pos_new#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6744186  0.69767442 0.70930233 0.6744186  0.60465116 0.61627907
 0.63953488 0.68604651 0.6        0.64705882]
Accuracy: 0.655 (0.037)


In [125]:
#9
X = bi_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6744186  0.69767442 0.70930233 0.6744186  0.60465116 0.61627907
 0.63953488 0.68604651 0.6        0.64705882]
Accuracy: 0.655 (0.037)


In [131]:
#10
X = bi_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6744186  0.69767442 0.70930233 0.6744186  0.60465116 0.61627907
 0.63953488 0.68604651 0.6        0.64705882]
Accuracy: 0.655 (0.037)


In [146]:
#11
X = tfidf_pos_new#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.65116279
 0.70930233 0.6744186  0.72941176 0.70588235]
Accuracy: 0.683 (0.036)


In [133]:
#12
X = tfidf_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.65116279
 0.70930233 0.6744186  0.72941176 0.70588235]
Accuracy: 0.683 (0.036)


In [135]:
#13
X = tfidf_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.74418605 0.62790698 0.68604651 0.65116279 0.65116279 0.65116279
 0.70930233 0.6744186  0.72941176 0.70588235]
Accuracy: 0.683 (0.036)


In [136]:
#14
X = wordvec_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.34883721 0.30232558 0.29069767 0.37209302 0.27906977 0.3255814
 0.31395349 0.39534884 0.28235294 0.74117647]
Accuracy: 0.365 (0.131)


In [137]:
#15
X = wordvec_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.34883721 0.30232558 0.29069767 0.38372093 0.27906977 0.3372093
 0.30232558 0.41860465 0.27058824 0.74117647]
Accuracy: 0.367 (0.132)


In [138]:
#16
X = wordvec_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.34883721 0.30232558 0.29069767 0.38372093 0.27906977 0.3372093
 0.30232558 0.41860465 0.27058824 0.74117647]
Accuracy: 0.367 (0.132)


In [139]:
#17
X = docvec_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.86046512 0.68604651 0.76744186 0.84883721 0.79069767 0.80232558
 0.76744186 0.75581395 0.84705882 0.81176471]
Accuracy: 0.794 (0.050)


In [22]:
#18
X = docvec_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.87209302 0.6744186  0.76744186 0.84883721 0.79069767 0.8372093
 0.77906977 0.73255814 0.87058824 0.78823529]
Accuracy: 0.796 (0.060)


In [141]:
#19
X = docvec_num_pos#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.87209302 0.6744186  0.77906977 0.84883721 0.79069767 0.8255814
 0.76744186 0.73255814 0.85882353 0.8       ]
Accuracy: 0.795 (0.058)


In [142]:
#20
X = pos_num#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6744186  0.61627907 0.60465116 0.59302326 0.59302326 0.59302326
 0.5        0.59302326 0.64705882 0.68235294]
Accuracy: 0.610 (0.049)


In [143]:
#21
X = all_features#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.77906977 0.74418605 0.76744186 0.72093023 0.72093023 0.69767442
 0.80232558 0.68604651 0.75294118 0.70588235]
Accuracy: 0.738 (0.036)


In [148]:

#experiment
X = pos_bi_count_vec#data
y = jhj_emoji_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.69767442 0.76744186 0.76744186 0.70930233 0.69767442 0.63953488
 0.68604651 0.69767442 0.69411765 0.77647059]
Accuracy: 0.713 (0.041)
